In [682]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from typing import List, Dict, Tuple
import copy
import time
import numpy as np
from joblib import Parallel, delayed
from itertools import combinations, product
from queue import PriorityQueue, Empty
from helpers import *

In [729]:
nodes, costs, D = read_data(filename="TSPA.csv")

Representation of moves:
1) Nodes - (node_old, node_new)
   - node_old is the one to be exchanged in the path
   - you can check if node is still in path
   - you can check if node_value_new is already used in path
   - when you create new path just brute force enumerate, it is still fast
2) Edges - ((edge1_start, edge1_end), (edge2_start, edge2_end))
   - 4 node indeces
   - you can easily check if edge still exists in path
   - you can check the opposite order easily

In [730]:
class Move:
    def __init__(self, delta_value: int, move_type: str, values: tuple, adjacent=None):
        self.delta = delta_value
        self.type = move_type
        self.values = values
        self.adjacent = adjacent
        self.reversed = False
    def __gt__(self, _):
        return True
    def __lt__(self, _):
        return False
    def create_path(self, path):
        if self.type == 'nodes':
            new_path = path[:]
            old_node, new_node = self.values
            old_node_idx = new_path.index(old_node)
            new_path[old_node_idx] = new_node
            return new_path
        elif self.type == 'edges':
            edge1, edge2 = self.values
            if self.reversed:
                n1 = path.index(edge1[1])
                n2 = path.index(edge2[1])
            else:
                n1 = path.index(edge1[0])
                n2 = path.index(edge2[0])
            if n1 < n2:
                return path[:n1] + [path[n1]] + [path[n2]] + path[n1+1:n2][::-1] + path[n2+1:]
            else:
                return path[n2+1:n1] + [path[n1]] + [path[n2]] + path[:n2][::-1] + path[n1+1:][::-1]
        else:
            raise ValueError("Invalid delta_type!")

In [754]:
def print_move(move, prefix='Putting'):
    print(f'{prefix} {move.type} move {move.values} with delta: {move.delta}')

def get_nodes_delta(old_node, new_node, path, D, costs, prev, next):
    old_dist = D[prev, old_node] + D[old_node, next]
    new_dist = D[prev, new_node] + D[new_node, next]
    delta = new_dist - old_dist + costs[new_node] - costs[old_node]
    return int(delta)

def get_edges_delta(edge1, edge2, D):
    return int(( D[edge1[0], edge2[0]] + D[edge1[1], edge2[1]] ) \
          - ( D[edge1[0], edge1[1]] + D[edge2[1], edge2[0]] ))

def nodes_inter(path, D, costs):
    og_nodes = set(path)
    nodes = set(list(range(D.shape[0])))
    changes = product(og_nodes, nodes - og_nodes)
    moves = []
    for change in changes:
        old_node, new_node = change
        index_of_old_node = path.index(old_node)
        prev = int(path[(index_of_old_node - 1) % len(path)])
        next = int(path[(index_of_old_node + 1) % len(path)])
        delta = get_nodes_delta(old_node, new_node, path, D, costs, prev, next)
        move = Move(delta, 'nodes', (old_node, new_node), adjacent=(prev, next))
        moves.append(move)
    return moves

def edges_intra(path, distance_matrix):
    pairs1 = np.column_stack((path[:-1], path[1:]))
    cyclic_pair = np.array([path[-1], path[0]])
    all_edges = np.vstack((pairs1, cyclic_pair))
    edge_swaps = combinations(all_edges, 2)
    moves = []
    for swap in edge_swaps:
        edge1, edge2 = [tuple(arr) for arr in swap]
        if edge1[1] == edge2[0] or edge1[0] == edge2[1]:
            continue
        delta1 = get_edges_delta(edge1, edge2, distance_matrix)
        delta2 = get_edges_delta(edge1, edge2[::-1], distance_matrix)
        move1 = Move(delta1, 'edges', (edge1, edge2))
        move2 = Move(delta2, 'edges', (edge1, edge2[::-1]))
        moves.extend([move1, move2])
    return moves

def get_initial_moves(current_path, distance_matrix, costs):
        moves_inter = nodes_inter(current_path, distance_matrix, costs)
        moves_intra = edges_intra(current_path, distance_matrix)
        return moves_inter + moves_intra

# Scenario 1 - edges are in solution in order or both in reversed order - Apply - correct or reversed order - result of operation the same but still need to distinghuish due to create_path function in Move
# Scenario 2 - edges are in path but one of them in the opposite order - leave but do not apply
# Scenario 3 - edges used by the move no longer exist in path - remove
# When checking status take into account the neighbourhood of the node when checking for nodes_exchange.
# If one of the neighbouring nodes changes, delta for main node also changes so in future iterations
# you need to consider that
def get_move_status(move, path):
    if move.type == 'nodes':
        old_node, new_node = move.values
        if old_node in path and new_node not in path:
            prev_adj, next_adj = move.adjacent
            old_node_idx = path.index(old_node)
            prev = path[(old_node_idx-1) % len(path)]
            next = path[(old_node_idx+1) % len(path)]
            if prev_adj == prev and next_adj == next:
                return "correct order"
        return "incorrect"
    else: # edges
        edge1, edge2 = move.values
        edge1_start, edge1_end = edge1
        edge2_start, edge2_end = edge2
        tmp_path = path + [path[0]]
        # check if both edge1 and edge2 are in path in order
        # and if either edge1 or edge2 are in path in opposite order
        e1_order, e2_order = False, False
        e1_opposite, e2_opposite = False, False
        for idx, node_val in enumerate(tmp_path):
            if idx == len(tmp_path)-1:
                break
            if edge1_start == node_val and edge1_end == tmp_path[idx+1]:
                e1_order = True
            if edge2_start == node_val and edge2_end == tmp_path[idx+1]:
                e2_order = True
            if edge1_end == node_val and edge1_start == tmp_path[idx+1]:
                e1_opposite = True
            if edge2_end == node_val and edge2_start == tmp_path[idx+1]:
                e2_opposite = True
        if e1_order and e2_order:
            return "correct order"
        elif e1_opposite and e2_opposite:
            return "reversed order"
        elif (e1_order and e2_opposite) or (e1_opposite and e2_order):
            return "partly reversed order"
        else:
            return "incorrect"

def get_best_move(moves, path):
    moves_to_retain = [] # [(delta, move), ...]
    choice = None
    while not moves.empty():
        delta, move = moves.get()
        status = get_move_status(move, path)
        if status == "correct order":
            move.reversed = False
            choice = move
            break
        elif status == "reversed order":
            move.reversed = True
            choice = move
            break
        elif status == "partly reversed order":
            moves_to_retain.append((delta, move))
            # you can't put the move back in here because of infinite loop
        else:
            continue
    for delta_move in moves_to_retain:
        moves.put(delta_move)
    return choice


In [755]:
def update_moves(moves: PriorityQueue, last_move, path, costs, D) -> PriorityQueue:
    if moves.empty():
        new_moves = get_initial_moves(path, D, costs)
        for move in new_moves:
            if move.delta < 0:
                moves.put((move.delta, move))
        return moves
    else:
        new_path = last_move.create_path(path)
        new_path.append(new_path[0])
        nodes_not_in_path = [x for x in range(len(D)) if x not in new_path]

        if last_move.type == 'nodes':
            old_node, new_node = last_move.values
            old_node_idx = path.index(old_node)
            prev = int(new_path[(old_node_idx - 1) % len(new_path)])
            next = int(new_path[(old_node_idx + 1) % len(new_path)])
            # moves for exchanging newly added node
            for node in nodes_not_in_path:
                exchange = (new_node, node)
                delta = get_nodes_delta(new_node, node, new_path, D, costs, prev, next)
                if delta < 0:
                    move = Move(delta, 'nodes', exchange, adjacent=(prev, next))
                    moves.put((delta, move))
            # when exchanging node you change only edges
            # moves for exchanging newly added edges
            new_edge1 = (prev, new_node)
            new_edge2 = (new_node , next)
            usable_edges = []
            # get all edges except for newly added ones
            for idx, node in enumerate(new_path):
                if idx == len(new_path)-1:
                    break
                next_node = new_path[idx+1]
                if node == new_node or next_node == new_node: # new edges
                    continue
                usable_edges.append((node, next_node))
            # for each usable edge in path, make a possible exchange
            # with each one of two newly introduced edges
            for tmp_edge in [new_edge1, new_edge2]:
                for edge in usable_edges:
                    exchanges = [
                        (tmp_edge, edge),
                        (tmp_edge, edge[::-1]),
                    ]
                    for edge1, edge2 in exchanges:
                        delta = get_edges_delta(edge1, edge2, D)
                        if delta < 0:
                            move = Move(delta, 'edges', (edge1, edge2))
                            moves.put((delta, move))
        else:
            # Add moves for new edges, to every other edge
            edge1, edge2 = last_move.values
            edge1_start, edge1_end = edge1
            edge2_start, edge2_end = edge2
            new_edge1 = (edge1_start, edge2_start)
            new_edge2 = (edge1_end, edge2_end)
            # when exchanging 2 edges, the neighbourhood of 4 nodes changes,
            # so you need to add nodes exchanges for each of them
            for changed_node in (edge1_start, edge2_start, edge1_end, edge2_end):
                node_idx = new_path.index(changed_node)
                prev = int(new_path[(node_idx - 1) % len(path)])
                next = int(new_path[(node_idx + 1) % len(path)])
                for node in nodes_not_in_path:
                    exchange = (changed_node, node)
                    delta = get_nodes_delta(changed_node, node, new_path, D, costs, prev, next)
                    if delta < 0:
                        move = Move(delta, 'nodes', exchange, adjacent=(prev, next))
                        moves.put((delta, move))

            usable_edges = []
            # get all edges except for newly added ones
            for idx, node in enumerate(new_path):
                if idx == len(new_path)-1:
                    break
                next_node = new_path[idx+1]
                # currently on edge - (node, next_node)
                if (node == new_edge1[0] and next_node == new_edge1[1]) or \
                   (node == new_edge2[0] and next_node == new_edge2[1]):
                    continue
                usable_edges.append((node, next_node))

            for tmp_edge in [new_edge1, new_edge2]:
                for edge in usable_edges:
                    exchanges = [
                        (tmp_edge, edge),
                        (tmp_edge, edge[::-1]),
                    ]
                    for edge1, edge2 in exchanges:
                        delta = get_edges_delta(edge1, edge2, D)
                        if delta < 0:
                            move = Move(delta, 'edges', (edge1, edge2))
                            moves.put((delta, move))
        return moves

In [759]:
def local_search_deltas(current_path, costs, D):
    moves = PriorityQueue()
    moves = update_moves(moves, None, current_path, costs, D)
    while True:
        best_move = get_best_move(moves, current_path)
        if best_move is None:
            break
        else:
            moves = update_moves(moves, best_move, current_path, costs, D)
            current_path = best_move.create_path(current_path)

    return current_path

initial_solution = list(random_sequence(D))
solution = local_search_deltas(initial_solution, costs, D)
print(evaluate(D, solution, costs))


78582


For nodes exchange you recalculate:
1. Edge exchanges for 2 newly introduced edges
2. Nodes exchanges for 2 surrounding nodes
3. Nodes exchanges for newly introduced node

For edges exchange you recalculate:
1. Edge exchanges for 2 newly introduces edges
2. Nodes exchanges for 4 nodes for which neighbourhood changed